In [1]:
%pip install langchain
%pip install transformers
%pip install sentence-transformers
%pip install pandas
%pip install -U langchain-community faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.0 MB/s eta 0:00:00


In [6]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import torch
import random
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import os
import pprint as pprint


# Set seed for vector database
random.seed(42)
torch.manual_seed(42)

# Check device
has_gpu = torch.cuda.is_available()
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(device)

if has_gpu:
    torch.cuda.manual_seed_all(42)

cpu


In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

# Extract gold labels and queries
eval_data = pd.read_csv('questions_answers.csv')
eval_data.rename(columns={eval_data.columns[0]: 'PMID'}, inplace=True)

gold_pmids = eval_data['PMID'].to_list()
eval_queries = eval_data['QUESTION'].to_list()

Mounted at /content/drive
/content/drive/MyDrive


In [7]:
# Load embeddings
model_id = 'BAAI/bge-base-en-v1.5'
#model_id = 'llmrails/ember-v1'
#model_id = 'intfloat/e5-base-v2'

file_path = model_id.replace("/", "_")

model_kwargs = {'device': device}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(model_name=model_id, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

new_db = FAISS.load_local(f"faiss_indices/{model_id}", embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# Select wide range of queries in the eval set

query_embeddings = np.array([embeddings.embed_query(query) for query in eval_data['QUESTION']])

# Compute cosine dissimilarity matrix
similarity_matrix = 1 - cosine_similarity(query_embeddings)

# Fill the diagonal with large negative values to avoid self-selection
np.fill_diagonal(similarity_matrix, -np.inf)

# Find indices of the least similar queries
dissimilarity_sums = similarity_matrix.sum(axis=1)
most_dissimilar_indices = np.argsort(dissimilarity_sums)[-8:]

# Select the most dissimilar queries and their PMIDs
selected_queries_and_pmids = eval_data[['QUESTION', 'PMID']].iloc[most_dissimilar_indices]

# Find the shortest and longest queries by character count and their PMIDs
shortest_query_index = eval_data['QUESTION'].str.len().idxmin()
longest_query_index = eval_data['QUESTION'].str.len().idxmax()
shortest_query_and_pmid = eval_data[['QUESTION', 'PMID']].iloc[shortest_query_index]
longest_query_and_pmid = eval_data[['QUESTION', 'PMID']].iloc[longest_query_index]


In [10]:
# Append the shortest and longest queries with their PMIDs to the selected queries
selected_queries_and_pmids = pd.concat([selected_queries_and_pmids, pd.DataFrame([shortest_query_and_pmid, longest_query_and_pmid])])

# Print the selected queries and PMIDs
pprint.pprint(selected_queries_and_pmids.values.tolist())

[['Is occupational outcome in bipolar disorder predicted by premorbid '
  'functioning and intelligence?',
  23527993],
 ['Is bilateral hearing loss associated with decreased nonverbal intelligence '
  'in US children aged 6 to 16 years?',
  24913183],
 ['Is cognitive ability in early adulthood associated with later suicide and '
  'suicide attempt : the role of risk factors over the life course?',
  22617391],
 ['Does parenting behavior at 2 years predict school-age performance at 7 '
  'years in very preterm children?',
  26616792],
 ['Is excess of runs of homozygosity associated with severe cognitive '
  'impairment in intellectual disability?',
  25232855],
 ['Does randomised trial of early neonatal hydrocortisone demonstrate '
  'potential undesired effects on neurodevelopment at preschool age?',
  26058477],
 ['Is prenatal Micronutrient Supplementation Associated with Intellectual '
  'Development of Young School-Aged Children?',
  26084366],
 ['Is maternal stress during pregnanc

In [19]:
gold_pmids = selected_queries_and_pmids['PMID'].to_list()
selected_queries = selected_queries_and_pmids['QUESTION'].to_list()

# Function to write the evaluation results to a text file
def write_evaluation_results(eval_data, gold_pmids, filename):
    with open(filename, 'w') as file:
        retriever = new_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

        file.write(f"Results for k=3\n")
        file.write("----------\n")

        for query, gold_label in zip(eval_data['QUESTION'], gold_pmids):
            top_k_results = retriever.get_relevant_documents(query)
            retrieved_pmids = [int(result.metadata['PMID']) for result in top_k_results]

            file.write(f"Query: {query}\n\n")
            for idx, result in enumerate(top_k_results):
                correct_mark = "(Correct)" if retrieved_pmids[idx] == gold_label else ""
                file.write(f"  Chunk: {result.page_content}... PMID: {retrieved_pmids[idx]} {correct_mark}\n\n")

            file.write("\n")  # Extra newline for readability
        file.write("\n\n")  # Extra newlines to separate sections for different k values


filename = f"retrieval_qualitative_evaluation_{file_path}.txt"
write_evaluation_results(selected_queries_and_pmids, gold_pmids, filename)
